In [1]:
%matplotlib inline

import sys, os, glob

import numpy as np
from numpy.linalg import norm

import scipy as sp
from scipy import stats
from scipy.misc import comb

from math import floor, ceil, log
import statistics as stat

import matplotlib.pyplot as plt

import codecs, re

from sympy import *

In [369]:
# Папка для сохранения графиков
images_folder = '../CourseWork/images/'

In [370]:
# Число Куранта, шаги по пространству-времени
cour = S('0.9')
h    = S('0.025')
dt   = cour * h

# Отрезок [0, L], считаем до времени T
L = 2.0
T = 1.4

# На сколько отрезков разбиваются промежутки [0, L] и [0, T]
M = int(L / h)
N = int(T / dt)

In [371]:
# Моменты времени, по которым планируется строить графики
# В случае неустойчивиго поведения или по каким-нибудь другим причинам
# список, конечно, можно изменять
time_moments = np.array([0, 1, 2, 3, 4, 5, 6]) * 0.2

# Три момента времени - для чёрно-белого графика
time_moments_reduced = np.array([0, 3, 6]) * 0.2

In [372]:
z = Symbol('z')

In [373]:
# Исследуемые функции
f1 = 1 - z
f2 = z

In [374]:
# Красный цвет
col1 = (255/255.0, 0/255.0, 0/255.0)

In [375]:
# Толщина линий, размер шрифта (для графиков)
fsz_tick = 36
fsz_legend = 34
fsz_label = 38
lw = 5
ms = 10

In [376]:
# Функция для построения графиков (количество моментов времени ограничено,
# макс примерно 255 / 30 ~ 8, потому что раскрашиваются)
def generate_plot(grid, name=None):
    fig = plt.figure(figsize=(25,15))

    start_color = list(col1)
    x = h * np.array(range(M+1))

    for i, tm in enumerate(time_moments):
        col = list(start_color)
        col[0] = (col[0]*255 - i*30) / 255.0
        col[2] = (col[2]*255 + i*10) / 255.0
        col = tuple(col)

        y = grid[int(tm / dt)]

        # plotting
        plt.plot(x, y, '-o', color=col, linewidth=lw, markersize=ms,
                label='t={0:.2f}'.format(tm))

    plt.xlabel('x', fontsize=fsz_label, fontname='Times New Roman', labelpad=20)
    plt.ylabel('u', fontsize=fsz_label, fontname='Times New Roman', labelpad=20)
    plt.xticks(fontsize=fsz_tick, fontname='Times New Roman')
    plt.yticks(fontsize=fsz_tick, fontname='Times New Roman')
    plt.legend(loc='best', fontsize=fsz_legend)
    plt.grid(True)

    plt.axis('tight')
    
    if (name is not None):
        fig.savefig(os.path.join(images_folder, name),
                    format='eps', bbox_inches='tight')

In [377]:
# Чёрно-белый график по трём моментам времени
def generate_plot_black_white(grid, name=None):
    fig = plt.figure(figsize=(25,15))
    
    linestyles = [':', '--', '-']
    x = h * np.array(range(M+1))

    for i, tm in enumerate(time_moments_reduced):
        y = grid[int(tm / dt)]

        # plotting
        plt.plot(x, y, linestyles[i] + 'o',
                 color='black', linewidth=lw,
                 markersize=ms,
                 label='t={0:.2f}'.format(tm))

    plt.xlabel('x', fontsize=fsz_label, fontname='Times New Roman', labelpad=20)
    plt.ylabel('u', fontsize=fsz_label, fontname='Times New Roman', labelpad=20)
    plt.xticks(fontsize=fsz_tick, fontname='Times New Roman')
    plt.yticks(fontsize=fsz_tick, fontname='Times New Roman')
    plt.legend(loc='best', fontsize=fsz_legend)
    plt.grid(True)

    plt.axis('tight')

    if (name is not None):
        fig.savefig(os.path.join(images_folder, name),
                    format='eps', bbox_inches='tight')

In [378]:
# Инициализация сетки
def grid_init(f, h, dt):
    grid = {t: np.array([0.0 for i in range(M+1)])
            for t in range(N+1)}
    
    # u(0, x)
    grid[0] = np.array([f.subs(z, i*h) for i in range(M+1)])
    
    # u(t, 0)
    for t in grid:
        grid[t][0] = f.subs(z, 0)
    
    return grid

In [379]:
# Обновление последней, M-ой, координаты на слое n
def outer_update(grid, n, h, dt):
    if (n >= N):
        return grid
    
    # grid[n][M]
    grid[n][M] = (
        grid[n-1][M] - dt / h * (grid[n-1][M]**2   / 2 - 
                                     grid[n-1][M-1]**2 / 2)
    )
    
    return grid

In [380]:
# Далее - серия функций-методов обновлений внутренних точек сетки

In [381]:
def inner_update_silly(grid, n, h, dt):
    if (n >= N):
        return grid
    
    grid[n][1:M-1+1] = (
        grid[n-2][1:M-1+1] - dt / h * grid[n-1][1:M-1+1] * 
          (grid[n-1][2:(M+1)] - grid[n-1][0:(M-2+1)])
    )
    
    return grid

In [382]:
def inner_update_lax(grid, n, h, dt):
    if (n >= N):
        return grid
    
    grid[n][1:M-1+1] = (
        0.5 * (grid[n-1][2:M+1] + grid[n-1][0:M-2+1]) - 
        dt / 2 / h * (grid[n-1][2:M+1]**2 / 2 -
                      grid[n-1][0:M-2+1]**2 / 2)
    )
    
    return grid

In [383]:
def inner_update_cir(grid, n, h, dt):
    if (n >= N):
        return grid
    
    u = grid[n-1][1:M-1+1]
    
    grid[n][1:M-1+1] = (
      u - dt / h * (
        1 * (u <  0) * (grid[n-1][2:M+1]**2 / 2 - grid[n-1][1:M-1+1]**2 / 2) +
        1 * (u >= 0) * (grid[n-1][1:M-1+1]**2 / 2 - grid[n-1][0:M-2+1]**2 / 2)
      )
    )
    
    return grid

In [384]:
def inner_update_lax_vendroff(grid, n, h, dt):
    if (n >= N):
        return grid
    
    ur = (
        0.5 * (grid[n-1][2:M+1] + grid[n-1][1:M-1+1]) - 
        dt / 2 / h * (grid[n-1][2:M+1]**2 / 2 - grid[n-1][1:M-1+1]**2 / 2)
    )
    
    ul = (
        0.5 * (grid[n-1][1:M-1+1] + grid[n-1][0:M-2+1]) - 
        dt / 2 / h * (grid[n-1][1:M-1+1]**2 / 2 - grid[n-1][0:M-2+1]**2 / 2)
    )
    
    grid[n][1:M-1+1] = (
        grid[n-1][1:M-1+1] - 
        dt / h * (ur**2 / 2 - ul**2 / 2)
    )
    
    return grid

In [385]:
def inner_update_maccormack(grid, n, h, dt):
    if (n >= N):
        return grid
    
    a = 1
    
    um = (
        grid[n-1][1:M-1+1] - 
        a * dt / h * (grid[n-1][1:M-1+1]**2 / 2 - grid[n-1][0:M-2+1]**2 / 2)
    )
    
    ur = (
        grid[n-1][2:M+1] -  
        a * dt / h * (grid[n-1][2:M+1]**2 / 2 - grid[n-1][1:M-1+1]**2 / 2)
    )
    
    grid[n][1:M-1+1] = (
        0.5 * (grid[n-1][1:M-1+1] + um) - 
        a * dt / h * (ur**2 / 2 - um**2 / 2)
    )
    
    return grid

### 1. 'Silly'

In [386]:
f = f1
grid = grid_init(f, h, dt)

In [ ]:
# u(1, x)
grid[1] = np.array([f.subs(z, i*h) * (1 - f.diff(z).subs(z, i*h) * dt)
           for i in range(M+1)])
grid = outer_update(grid, 1, h, dt)

for n in range(2, N+1):
    grid = inner_update_silly(grid, n, h, dt)
    grid = outer_update(grid, n, h, dt)

In [ ]:
file_name = '1.silly.cour{0}.f{1}{2}.eps'.format(
    ''.join(str('{0:.1f}'.format(cour)).split('.')),
    1,
    ''.join(str('{0:.3f}'.format(h)).split('.'))[1:]
)

generate_plot(grid, file_name)

print('Saved as ' + file_name)

### 2. Lax

In [389]:
f = f1
grid = grid_init(f, h, dt)

In [390]:
for n in range(1, N+1):
    grid = inner_update_lax(grid, n, h, dt)
    grid = outer_update(grid, n, h, dt)

In [ ]:
file_name = '2.lax.cour{0}.f{1}{2}.eps'.format(
    ''.join(str('{0:.1f}'.format(cour)).split('.')),
    1,
    ''.join(str('{0:.3f}'.format(h)).split('.'))[1:]
)

generate_plot(grid, file_name)

print('Saved as ' + file_name)

### 3. Courant, Isaacson, Rees (CIR)

In [392]:
f = f1
grid = grid_init(f, h, dt)

In [393]:
for n in range(1, N+1):
    grid = inner_update_cir(grid, n, h, dt)
    grid = outer_update(grid, n, h, dt)

In [ ]:
file_name = '3.cir.cour{0}.f{1}{2}.eps'.format(
    ''.join(str('{0:.1f}'.format(cour)).split('.')),
    1,
    ''.join(str('{0:.3f}'.format(h)).split('.'))[1:]
)

generate_plot(grid, file_name)

print('Saved as ' + file_name)

### 4. Lax-Vendroff

In [395]:
f = f2
grid = grid_init(f, h, dt)

In [ ]:
for n in range(1, N+1):
    grid = inner_update_lax_vendroff(grid, n, h, dt)
    grid = outer_update(grid, n, h, dt)

In [ ]:
file_name = '4.lax-vendroff.cour{0}.f{1}{2}.eps'.format(
    ''.join(str('{0:.1f}'.format(cour)).split('.')),
    2,
    ''.join(str('{0:.3f}'.format(h)).split('.'))[1:]
)

generate_plot(grid, file_name)

print('Saved as ' + file_name)

### 5. MacCormack

In [398]:
f = f2
grid = grid_init(f, h, dt)

In [ ]:
for n in range(1, N+1):
    grid = inner_update_maccormack(grid, n, h, dt)
    grid = outer_update(grid, n, h, dt)

In [ ]:
file_name = '5.maccormack.cour{0}.f{1}{2}.eps'.format(
    ''.join(str('{0:.1f}'.format(cour)).split('.')),
    2,
    ''.join(str('{0:.3f}'.format(h)).split('.'))[1:]
)

generate_plot(grid, file_name)

print('Saved as ' + file_name)